# Poisson Equation

**Goal:** give details for solving the Poisson equation numerically for an arbitray potential. 

The Poisson equation reads
$$
\nabla^2 \phi(x) = 4\pi \rho(x)
$$

In [1]:
using FFTW
using GLMakie
using SpecialFunctions

## Fast-Fourier Transform

For periodic boundary condition it is conveniant to solve the Poisson equation using Fourier transform.

We want to solve the equation
$$
\nabla^2 \phi = 4\pi G \rho
$$
using the fast-Fourier transform.

In momentum space the equation simply becomes
$$
\phi(\vec{p}) = - \frac{4\pi G}{\vec{p}^{\,2}} \rho(\vec{p}) 
$$
which give the direct solution for $\phi(x)$
$$
\phi(x) = - \int \frac{d^dp}{(2\pi)^d} \frac{4\pi G}{\vec{p}^{\,2}} \rho(\vec{p})
$$

In [211]:
function poisson_solve_3d(ρ, (; Nx, dx), (; Ny, dy), (; Nz, dz))
    kx = 2π * fftfreq(Nx, 1 / dx)
    ky = 2π * fftfreq(Ny, 1 / dy)
    kz = 2π * fftfreq(Nz, 1 / dz)

    ρhat = fft(ρ)

    uhat = zeros(Complex{Float64}, Nx, Ny, Nz)

    for (i, kxi) in enumerate(kx), (j, kyj) in enumerate(ky), (k, kzk) in enumerate(kz)
        if kxi == 0 && kyj == 0 && kzk == 0
            uhat[i, j, k] = 0
        else
            uhat[i, j, k] = -4π * ρhat[i, j, k] / (kxi^2 + kyj^2 + kzk^2)
        end
    end

    u = real(ifft(uhat))

    return u
end

poisson_solve_3d (generic function with 1 method)

Setup the discretization and the initial problem

In [274]:
xmin = 0
xmax = 1
Nx = 1000

ymin = 0
ymax = 1
Ny = 100

zmin = 0
zmax = 1
Nz = 100

xl = range(xmin, xmax, Nx)
yl = range(ymin, ymax, Ny)
zl = range(zmin, zmax, Nz)

dx = xl[2] - xl[1]
dy = yl[2] - yl[1]
dz = zl[2] - zl[1]

x0 = xmin + (xmax - xmin) / 2
y0 = ymin + (ymax - ymin) / 2

ρ = zeros(Nx, Ny, Nz)
α = 0.0001 # How close the density distribution matches a delta function.
for (i, x) in enumerate(xl), (j, y) in enumerate(yl), (k, z) in enumerate(zl)
    # ρ[i, j, k] = exp(-((x - x0)^2 + (y - y0)^2) / α) / (α * π)
    ρ[i, j, k] = exp(-(x - x0)^2 / α) / sqrt(α * π)
end

Plot of the density distribution

In [1]:
# iz = Nz ÷ 2
iz = 2
contourf(ρ[:, :, iz])

UndefVarError: UndefVarError: `ρ` not defined

Solve the problem and plot the solution

In [275]:
u = poisson_solve_3d(ρ, (; Nx, dx), (; Ny, dy), (; Nz, dz));

In [281]:
uana = zeros(Nx, Ny, Nz)
for (i, x) in enumerate(xl), (j, y) in enumerate(yl), (k, z) in enumerate(zl)
    uana[i, j, k] = -1 / sqrt((x - x0)^2 + (y - y0)^2)
    # uana[i, j, k] = -1 / sqrt((x - x0)^2)
end

iy = div(Ny, 2)
iz = 3

f = lines(xl, u[:, iy, iz], color=:blue)
lines!(xl, uana[:, iy, iz], color=:red)
ylims!(low=-2)
f

In [201]:
f, ax = surface(xl, yl, u[:, :, iz]; axis=(; type=Axis3))
surface!(xl, yl, uana[:, :, iz])
zlims!(-20, 1)
f

Plot the Fourier transform of $\rho$.

In [101]:
ωx = fftfreq(Nx, 1 / dx) |> fftshift
ωy = fftfreq(Ny, 1 / dy) |> fftshift

ρhat = fft(ρ)
ρplot = fftshift(ρhat) .|> abs

iz = 6

fig = Figure()
ax = Axis(fig[1, 1])
contourf!(xl, yl, ρ[:, :, iz])
ax = Axis(fig[1, 2])
contourf!(ωx, ωy, ρplot[:, :, iz])
fig

## Trying a pseudo-3D method

In [2]:
function poisson_solve_3d(ρ, (; Nx, dx), (; Ny, dy); ϵ=0.1, G=1)
    kx = 2π * fftfreq(Nx, 1 / dx)
    ky = 2π * fftfreq(Ny, 1 / dy)

    ρhat = fft(ρ)

    uhat = zeros(Complex{Float64}, Nx, Ny)

    for (i, kxi) in enumerate(kx), (j, kyj) in enumerate(ky)
        if kxi == 0 && kyj == 0
            uhat[i, j] = 0
        else
            uhat[i, j] = -4π * G * ρhat[i, j] / (kxi^2 + kyj^2 + ϵ^2)
        end
    end

    u = real(ifft(uhat))

    return u
end

poisson_solve_3d (generic function with 1 method)

Setup the discretization and the initial problem

In [38]:
xmin = 0
xmax = 1
Nx = 100

ymin = 0
ymax = 1
Ny = 100

xl = range(xmin, xmax, Nx)
yl = range(ymin, ymax, Ny)

dx = xl[2] - xl[1]
dy = yl[2] - yl[1]

x0 = xmin + (xmax - xmin) / 2
y0 = ymin + (ymax - ymin) / 2

ρ = zeros(Nx, Ny)
uana = zeros(Nx, Ny)

# alpha = 0.05 # How close the density distribution matches a delta function.
σ = 0.05

for (i, x) in enumerate(xl), (j, y) in enumerate(yl)
    r = sqrt((x - x0)^2 + (y - y0)^2)

    # ρ[i, j] = exp(-r^2 / alpha) / (alpha * π)
    # uana[i, j] = -1 / sqrt((x - x0)^2 + (y - y0)^2)

    ρ[i, j] = 1 / (σ^3 * (2π)^(3 / 2)) * exp(-r^2 / 2σ^2)
    uana[i, j] = -1 / r * erf(r / √2σ)
end

In [20]:
heatmap(xl, yl, ρ)

In [45]:
u = poisson_solve_3d(ρ, (; Nx, dx), (; Ny, dy); ϵ=10);

In [48]:
iy = div(Ny, 2)

f = lines(xl, u[:, iy], color=:blue)
lines!(xl, uana[:, iy], color=:red)
f